In [1]:
from calculate_colocations import get_pos_filtered_colloc_from_corpus_list, get_colloc_from_corpus_list,get_clean_lemm_list
import pandas as pd
from tqdm import tqdm
from collections import Counter

import json

In [2]:
texts_lenta = pd.read_csv("C://Autotutor//improved_approach//articles//music_lenta.csv")

texts_lenta.iloc[0]['text']

'Расходы на проведение музыкального конкурса «Евровидение-2017» в Киеве оказались предельно высокими. Об этом в четверг, 11 мая, пишет украинское издание «Вести». «Украинцев возмутили запредельные гонорары артистов и ведущих шоу, то, что почти все тендеры проводились в сжатые сроки по сокращенной процедуре, а также явно лишние, неоправданные траты в стране, где идет война (речь идет о вооруженном конфликте в Донбассе — прим. «Ленты.ру», а большой части населения не хватает денег даже на оплату коммуналки», — говорится в заметке. По подсчетам газеты, на соревнование было потрачено 870 миллионов гривен (33 миллиона долларов). «Ровно столько закладывали в проект госбюджета Украины 2017 года на дотации государственными угольным шахтам (частичное покрытие разницы в себестоимости угля)», — пишет издание. По мнению авторов, пять миллионов гривен (190 тысяч долларов), потраченные на белое полотно, которым задрапировали «недострой рядом с местом проведения конкурса», «явно можно было бы потрати

In [29]:
clean_lemm_list = get_clean_lemm_list(list(texts_lenta['text']),lang = 'rus')


 21%|████████████████▊                                                               | 105/500 [00:34<02:11,  3.01it/s]

KeyboardInterrupt: 

In [22]:
'на' in clean_lemm_list[1:10]

True

In [18]:
['на', 'проведение'] in clean_lemm_list[1:10]

False

In [4]:
unigramm_freq = Counter(clean_lemm_list)


In [34]:
with open ("music_unigr_freq.json","w", encoding = "utf-8") as f:
    json.dump(unigramm_freq, f, indent = 4, ensure_ascii = False)

In [3]:
bigramFreqTable, trigramFreqTable, quadgram_freq, filtered_bi, filtered_tri, bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable =get_pos_filtered_colloc_from_corpus_list(list(texts_lenta['text']),"rus")


100%|██████████████████████████████████████████████████████████████████████████████| 5099/5099 [38:08<00:00,  2.23it/s]


In [8]:
def cean_pos_tags(df, ngramm_name, handle_percent = 1):
    clean_words = []
    search_range =int(handle_percent * len(df))
    for i in tqdm(range(search_range)):
        posed_ngramm = df.iloc[i][ngramm_name]
        clean_ngramm = []
        for w in posed_ngramm:
            word = w.split("_")[0]
            clean_ngramm.append(word)
        clean_ngramm = tuple(clean_ngramm)
        clean_words.append(clean_ngramm)
    
    data = {"ngramm":clean_words,"freq":df['freq'][:search_range]}
    clean_df = pd.DataFrame(data) 
    return clean_df
bigramFreqTable_clean = cean_pos_tags(bigramFreqTable,'bigram', handle_percent = 0.3)
trigramFreqTable_clean = cean_pos_tags(trigramFreqTable, "trigram", handle_percent = 0.25)
quadgram_freq_clean = cean_pos_tags(quadgram_freq, "quadgramF", handle_percent = 0.2)
filtered_bi_clean = cean_pos_tags(filtered_bi,'bigram')
filtered_tri_clean = cean_pos_tags(filtered_tri,'trigram')

100%|██████████████████████████████████████████████████████████████████████████| 96474/96474 [00:26<00:00, 3598.93it/s]


In [10]:
bigramFreqTable_clean.to_dict("split")

{'columns': ['freq', 'ngramm'],
 'data': [[41, ('о', 'это')],
  [32, ('то', 'что')],
  [31, ('один', 'из')],
  [25, ('отмечать', 'что')],
  [22, ('во', 'время')],
  [21, ('о', 'то')],
  [20, ('2013', 'год')],
  [19, ('концерт', 'в')],
  [19, ('входить', 'в')],
  [19, ('по', 'слово')],
  [18, ('а', 'также')],
  [18, ('заявлять', 'что')],
  [18, ('в', 'киев')],
  [17, ('в', 'россия')],
  [16, ('2012', 'год')],
  [16, ('выходить', 'в')],
  [16, ('выступать', 'в')],
  [16, ('это', 'сообщать')],
  [15, ('год', 'в')],
  [15, ('проходить', 'в')],
  [15, ('2014', 'год')],
  [14, ('участие', 'в')],
  [14, ('в', 'конец')],
  [14, ('2015', 'год')],
  [14, ('что', 'он')],
  [14, ('в', 'который')],
  [14, ('это', 'в')],
  [13, ('в', 'тот')],
  [13, ('в', 'частность')],
  [13, ('принимать', 'участие')],
  [13, ('год', 'она')],
  [12, ('в', 'свой')],
  [12, ('на', 'сцена')],
  [12, ('в', 'он')],
  [12, ('он', 'слово')],
  [12, ('на', 'свой')],
  [12, ('в', 'москва')],
  [12, ('вместе', 'с')],
  [11, 

In [9]:
def get_freq_colloc_dict(ngrm_lis):
    freq_colloc_dict = {'2':{},'3':{},'4':{}}
    for ngramm_df in ngrm_lis:
        dct = ngramm_df.to_dict("split")
        #print(dct['data'][0][1],len(dct['data'][0][1]))
        words_len = len(dct['data'][0][1])
        for el in dct['data']:
            ngramm_raw = ''
            for el_i in el[1]:
                ngramm_raw += el_i + ' '
            ngramm_raw = ngramm_raw.strip()
            #print(ngramm_raw,words_len )
            freq_colloc_dict[str(words_len)][ngramm_raw] = el[0]
        
    return freq_colloc_dict
ngramms_list =[bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean]
freq_colloc_dict = get_freq_colloc_dict(ngramms_list)

In [34]:
trigramPMITable.head(10)

,trigram,PMI
0,"(così, fan, tutte)",28.881090
1,"(эйч, пи, бакстер)",27.770058
2,"(лан, дель, ря)",27.524946
3,"(hot, chili, peppers)",27.258159
4,"(o, sole, mio)",27.185096
5,"(magna, carta, holy)",27.091986
6,"(black, eyed, peas)",26.939983
7,"(spiders, from, mars)",26.881090
8,"(carta, holy, grail)",26.676949
9,"(red, hot, chili)",26.606083


In [24]:
filtered_bi_clean.head()

,freq,ngramm
1186,11,"(народный, артист)"
1965,10,"(лидер, группа)"
404,9,"(новый, альб)"
416,8,"(студийный, альб)"
2987,7,"(студийный, альбом)"


In [10]:
def get_united_numeric_colloc_base (numeric_calc_collocations_list, collocations_by_freq_dict):#все кроме фильтрованных по частям речи
    
    overall_colloc_json = {'2':{},'3':{},'4':{}}
    for colloc in numeric_calc_collocations_list:
        colloc_len = len(colloc)
        for key in list(colloc.keys()):
            if 'gram' in key:
                ngramm_name = key
            else:
                freq_name = key
        print(freq_name)
        n_of_words = str(len(colloc.iloc[0][ngramm_name]))
        search_range = colloc_len
        """
        if n_of_words == str(2):
            search_range = int(colloc_len * 0.4)
        elif n_of_words == str(3):
            search_range = int(colloc_len * 0.3)
        elif n_of_words == str(4):
            search_range = int(colloc_len * 0.2)
        """
        print(n_of_words)
        for index in tqdm(range(search_range)):
            collocation_element = colloc.iloc[index][ngramm_name]
            str_el = ''
            for el in collocation_element:
                str_el += el + ' '
            str_el = str_el.strip()
            if str_el in collocations_by_freq_dict[n_of_words]:
                overall_colloc_json[n_of_words][str_el] = collocations_by_freq_dict[n_of_words][str_el] 
            else:
                #print(str_el,"is not in overallcolloc",colloc.iloc[index][freq_name])
                overall_colloc_json[n_of_words][str_el] = int(colloc.iloc[index][freq_name])
            """
            if str_el in overall_colloc_json[n_of_words]:
                overall_colloc_json[n_of_words][str_el] *= int(colloc.iloc[index][freq_name])
            else:
                overall_colloc_json[n_of_words][str_el] = int(colloc.iloc[index][freq_name])
            """
            #print(str_el)
        #print("========")
    return overall_colloc_json
#ngramms_list = [bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean, bigramPMITable, trigramPMITable, quadragramPMITable]
ngramms_list = [bigramPMITable, trigramPMITable, quadragramPMITable,filtered_bi_clean,filtered_tri_clean]

log_collocations_vs_freq = get_united_numeric_colloc_base(ngramms_list,freq_colloc_dict)

PMI
2


100%|██████████████████████████████████████████████████████████████████████████| 13851/13851 [00:03<00:00, 3533.27it/s]


PMI
3


100%|██████████████████████████████████████████████████████████████████████████| 15890/15890 [00:04<00:00, 3581.18it/s]


PMI
4


100%|██████████████████████████████████████████████████████████████████████████| 22584/22584 [00:06<00:00, 3521.26it/s]


freq
2


100%|████████████████████████████████████████████████████████████████████████| 103948/103948 [00:47<00:00, 2183.92it/s]


freq
3


100%|██████████████████████████████████████████████████████████████████████████| 96474/96474 [00:45<00:00, 2132.00it/s]


In [40]:
log_collocations_vs_freq["2"]

{'depeche mode': 12,
 'mercury prize': 11,
 'spice girls': 10,
 'foo fighters': 13,
 'правоохранительный орган': 12,
 'led zeppelin': 16,
 'океан ельзи': 1,
 'джей зи': 12,
 'тейлор свифт': 11,
 'радиостанция говорит': 9,
 'курт кобейн': 10,
 'де лусия': 6,
 'next day': 18,
 'king crimson': 22,
 'дональд трамп': 11,
 'rolling stones': 20,
 'пол маккартни': 3,
 'иосиф кобзон': 12,
 'депутат госдума': 10,
 'роберт плант': 2,
 'юлия самойлов': 12,
 'примечание лентыру': 26,
 'произведение symphonic': 13,
 'риа новость': 23,
 'юрист игорь': 12,
 'со сторона': 10,
 'факт плагиат': 15,
 'rolling stone': 12,
 'дидье маруани': 17,
 '': 19,
 'жестокий любовь': 2,
 'социальный сеть': 13,
 'symphonic space': 14,
 'канье уэст': 4,
 'художественный руководитель': 10,
 'защита авторский': 11,
 'любовь переработка': 11,
 'служба безопасность': 3,
 'space дидье': 13,
 'шоу бизнес': 14,
 'игорь трунов': 14,
 'филипп киркоров': 21,
 'space dream': 1,
 'уголовный дело': 11,
 'би 2': 3,
 'зал слава': 3,
 

In [11]:
with open ("music_smart_colloc_freq_test.json","w", encoding = "utf-8") as f:
    json.dump(log_collocations_vs_freq, f, indent = 4, ensure_ascii = False)